# Inporting Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import librosa
import librosa.display
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical# Hot encoding y
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.advanced_activations import LeakyReLU
import keras
import h5py
from sklearn.model_selection import KFold
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.optimizers import SGD
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.layers import BatchNormalization
import warnings
warnings.filterwarnings("ignore")

# Importing the data

In [ ]:
Xdata = np.load('dev_data.npy',allow_pickle=True,encoding='bytes')
Ydata = np.load('dev_labels.npy',allow_pickle=True,encoding='bytes')

# Alignment of data

In [ ]:
def align(arr,yarr, k):
    k21 = 2*k+1
    new_arr = np.zeros((arr.shape[0]-k21,k21,40))
    for i in range(arr.shape[0]-k21):
        new_arr[i,:,:] = arr[i:i+k21,:]
        new_yarr = yarr[k:yarr.shape[0]-k-1,]
    return new_arr, new_yarr

In [ ]:
xdata, ydata = align(Xdata, Ydata,6)

In [ ]:
Xdata = xdata.reshape((xdata.shape[0], xdata.shape[1], xdata.shape[2],1))

In [ ]:
Xdata.shape

In [ ]:
lb = LabelEncoder()
Ydata = to_categorical(lb.fit_transform(ydata))

In [ ]:
xdata, xtest, ydata, ytest = train_test_split(Xdata, Ydata, test_size=0.25, random_state=42)
Xdata = []
Ydata = []

# Model defining

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(13, 40,1)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(138, activation='softmax'))
    # compile model
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(xdata, ydata, epochs=30, batch_size=2048, validation_data=(xtest, ytest), verbose=1)

In [ ]:
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']# Set figure size.
plt.figure(figsize=(12, 8))# Generate line plot of training, testing loss over epochs.
plt.plot(train_accuracy, label='Training Accuracy', color='#185fad')
plt.plot(val_accuracy, label='Validation Accuracy', color='orange')# Set title
plt.title('Training and Validation Accuracy by Epoch', fontsize = 25)
plt.xlabel('Epoch', fontsize = 18)
plt.ylabel('Categorical Crossentropy', fontsize = 18)
plt.xticks(range(0,30,5), range(0,30,5))
plt.legend(fontsize = 18);
plt.savefig('accuracy.png')

In [ ]:
model.save("final_modelV3.h5")

# loading model

In [ ]:
model = load_model('final_modelV2.h5', compile = True)
testx = np.load('test_data.npy',allow_pickle=True,encoding='bytes')

# Aligning the test data

In [ ]:
def align_test(arr, k):
    k21 = 2*k+1
    new_arr = np.zeros((arr.shape[0],k21,40))
    for i in range(k, arr.shape[0]-k):
        new_arr[i,:,:] = arr[i-k:i+k+1,:]
    for j in range(0,k):
        new_arr[j,:,:] = new_arr[j+k,:,:]
    for l in range(arr.shape[0]-k,arr.shape[0]):
        new_arr[l,:,:] = new_arr[l-k,:,:]
    return new_arr

In [ ]:
def test_data(arr,k):
    arr1 = align_test(arr,k)
    arr2 = xtest.reshape((arr1.shape[0], arr1.shape[1], arr1.shape[2],1))
    predictions = model.predict(arr2,batch_size=32)
    classes = np.argmax(predictions, axis = 1)
    return classes

# Predictions

In [ ]:
predictions = test_data(testx, 6)
predict = pd.DataFrame({"label": predictions,})

predict.to_csv('Speech_Recognition.csv', index_label="id")